In [1]:
from config_for_baseline import *

In [11]:
pip install -r requirments.txt 

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirments.txt'
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [35]:
import torchvision.transforms as transforms
from dataset import ImageClassificationDataset
from jetbot import Camera
camera = Camera.instance()

TASK = 'sign'
CATEGORIES = ['body','background']
DATASETS = ['body']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])


datasets = {}

for name in DATASETS:
    datasets[name] = ImageClassificationDataset(TASK +'_'+ name , CATEGORIES ,TRANSFORMS)
    print(name)

print("{} CATEGORIES {} TASK define ".format(CATEGORIES, TASK))



body
['body', 'background'] CATEGORIES sign TASK define 


In [37]:
import ipywidgets
import traitlets
from IPython.display import display
#from jetcam.utils import bgr8_to_jpeg
from jetbot import bgr8_to_jpeg
#import dataset as datasets

# initialize active dataset
dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')
save_widget = ipywidgets.Button(description='add')

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

# save image for category and update counts
def save(c):
    dataset.save_entry(camera.value, category_widget.value)
    count_widget.value = dataset.get_count(category_widget.value)
save_widget.on_click(save)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget]), dataset_widget, category_widget, count_widget, save_widget
])

# display(data_collection_widget)
print("data_collection_widget created")

data_collection_widget created


In [38]:
all_widget = ipywidgets.VBox([
    ipywidgets.HBox([data_collection_widget])
])

display(all_widget)